In [13]:
import time
from IPython.display import clear_output
import copy
import random
import math

# Define players
PLAYER_X = 'X'
ENEMY_O = 'O'

class Node:
    def __init__(self, state, parent=None, initial_state=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.wins = 0
        self.initial_state = initial_state

    def is_fully_expanded(self):
        return len(self.children) == len(self.untried_actions())

    def untried_actions(self):
        return [(i, j) for i in range(len(self.state)) for j in range(len(self.state[0])) if self.state[i][j] == ' ']

    def select_child(self):
        return max(self.children, key=lambda x: x.wins/x.visits + math.sqrt(2*math.log(self.visits)/x.visits))

    def add_child(self, action):
        state = copy.deepcopy(self.state)
        state[action[0]][action[1]] = PLAYER_X if self.state == self.initial_state else ENEMY_O
        child = Node(state, self, self.initial_state)
        self.children.append(child)
        return child

    def update(self, result):
        self.visits += 1
        self.wins += result

class TicTacToe:
    def __init__(self, board_size=6):
        self.board_size = board_size
        self.board = [[' ' for _ in range(board_size)] for _ in range(board_size)]
        self.initial_state = copy.deepcopy(self.board)

    def select_difficulty(self):
        while True:
            print("Select difficulty:")
            print("1. Easy (few simulations)")
            print("2. Medium (moderate simulations)")
            print("3. Hard (many simulations)")
            choice = input("Enter your choice (1/2/3): ")
            if choice in ['1', '2', '3']:
                if choice == '1':
                    return 100
                elif choice == '2':
                    return 500
                else:
                    return 1000
            else:
                print("Invalid choice. Please select 1, 2, or 3.")

    def evaluate_board(self):
        # Evaluate the board based on Tic Tac Toe rules
        for row in range(self.board_size):
            for col in range(self.board_size):
                if self.board[row][col] != ' ':
                    symbol = self.board[row][col]

                    # Check horizontal
                    if col + 3 < self.board_size and all(self.board[row][col + i] == symbol for i in range(4)):
                        return symbol

                    # Check vertical
                    if row + 3 < self.board_size and all(self.board[row + i][col] == symbol for i in range(4)):
                        return symbol

                    # Check diagonal (top-left to bottom-right)
                    if row + 3 < self.board_size and col + 3 < self.board_size and all(self.board[row + i][col + i] == symbol for i in range(4)):
                        return symbol

                    # Check diagonal (top-right to bottom-left)
                    if row + 3 < self.board_size and col - 3 >= 0 and all(self.board[row + i][col - i] == symbol for i in range(4)):
                        return symbol

        # If no winner, return None
        return None

    def is_board_full(self):
        for row in self.board:
            if ' ' in row:
                return False
        return True

    def player_x_move(self):
        while True:
            try:
                col = int(input(f"Enter column (1-{self.board_size}): ")) - 1
                if 0 <= col < self.board_size:
                    row = int(input(f"Enter row (1-{self.board_size}): ")) - 1
                    if 0 <= row < self.board_size:
                        if self.board[row][col] == ' ':
                            return row, col
                        else:
                            print("This position is already occupied. Try again.")
                    else:
                        print(f"Row value should be between 1 and {self.board_size}.")
                else:
                    print(f"Column value should be between 1 and {self.board_size}.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    def enemy_o_move(self, simulations):
        root = Node(self.board, initial_state=self.initial_state)
        for _ in range(simulations):
            node = root
            state = copy.deepcopy(self.board)

            # Selection
            while node.children != [] and node.untried_actions() == []:
                node = node.select_child()
                state = copy.deepcopy(node.state)

            # Expansion
            if node.untried_actions() != []:
                action = random.choice(node.untried_actions())
                state[action[0]][action[1]] = ENEMY_O if node.state == self.board else PLAYER_X
                node = node.add_child(action)

            # Simulation
            while self.evaluate_board() is None and not self.is_board_full():
                actions = [(i, j) for i in range(self.board_size) for j in range(self.board_size) if state[i][j] == ' ']
                if not actions:
                    break  # No available actions, terminate simulation
                action = random.choice(actions)
                state[action[0]][action[1]] = PLAYER_X if state == self.board else ENEMY_O

            # Backpropagation
            result = self.evaluate(state)
            while node is not None:
                node.update(result)
                node = node.parent

        return max(root.children, key=lambda x: x.visits).state

    def evaluate(self, board):
        winner = self.evaluate_board()
        if winner == ENEMY_O:
            return 1
        elif winner == PLAYER_X:
            return -1
        else:
            return 0

    def print_board(self):
        transposed_board = [[' ' for _ in range(self.board_size)] for _ in range(self.board_size)]

        for i in range(self.board_size):
            for j in range(self.board_size):
                transposed_board[j][i] = self.board[i][j]

        # Print top horizontal line
        print('+' + '-' * (self.board_size * 5 - 7) + '+')

        for row in transposed_board:
            print('| ' + ' | '.join(row) + ' |')
            print('+' + '-' * (self.board_size * 5 - 7) + '+')

    def start_game(self):
        simulations = self.select_difficulty()

        while not self.is_board_full():
            clear_output(wait=True)
            self.print_board()

            row, col = self.player_x_move()
            self.board[row][col] = PLAYER_X

            winner = self.evaluate_board()
            if winner:
                clear_output(wait=True)
                self.print_board()
                print(f"{winner} wins!")
                break

            if self.is_board_full():
                clear_output(wait=True)
                self.print_board()
                print("It's a tie!")
                break

            self.board = self.enemy_o_move(simulations)

            winner = self.evaluate_board()
            if winner:
                clear_output(wait=True)
                self.print_board()
                print(f"{winner} wins!")
                break

            time.sleep(1)

game = TicTacToe()
game.start_game()


+-----------------------+
| X | X | X | X |   |   |
+-----------------------+
|   |   |   |   |   |   |
+-----------------------+
|   |   |   | O |   |   |
+-----------------------+
|   |   | O |   |   |   |
+-----------------------+
|   |   |   |   |   |   |
+-----------------------+
|   |   | O |   |   |   |
+-----------------------+
X wins!
